In [ ]:
#### Part 1 - Summary statistics and plot
# Import relevant packages 
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

# Import dataset 
os.chdir('/Users/jessicalindsay/Downloads')
filename = 'pmp_takehome_2019.csv'
df = pd.read_csv(filename)

# Create new variable for each location's monthly profit, by substracting total costs from gross revenue
monthly_profit = df.loc[:,'Gross revenue'] - df.loc[:,'Fixed cost'] - df.loc[:,'Variable cost'] - df.loc[:, 'Rental cost']
df['Profit'] = monthly_profit

# Calculate annual profit for each location by summing monthly profits
df_annual_profit = df.groupby('Location number').agg({'Profit':'sum'})

# Calculate mean and median values for annual profit over all stores
mean_profit = df_annual_profit.mean()
median_profit = df_annual_profit.median()
print(mean_profit)
print(median_profit)

# One property of the normal distribution is that the mean and median are equal. 
# Since the mean profit is significantly larger than the median profit, the profit margin for the stores is not normally distributed.
# In order to confirm that the data is not normally distributed, a histogram is a useful visualization tool 
prof = df_annual_profit.to_numpy()
plt.hist(prof,bins=15)
plt.gca().set(title='Annual Profit Histogram',xlabel='Annual Profit ($)', ylabel='Frequency');

# Plot monthly and quarterly aggregate revenue across all stores
#plot separately
#then together
# Calculate monthly revenue for all locations
monthly_rev = df.loc[:, ['Month', 'Gross revenue']]
monthly_rev_total = monthly_rev.groupby('Month').agg({'Gross revenue':'sum'})

# Calculate quarterly revenue
def quarter_df (df) :
    if df['Month']==1 or df['Month']==2 or df['Month']==3 :
        return 1
    elif df['Month']==4 or df['Month']==5 or df['Month']==6 :
        return 2
    elif df['Month']==7 or df['Month']==8 or df['Month']==9 :
        return 3
    else :
        return 4
    
df['Quarter'] = df.apply(quarter_df, axis = 1)

quarterly_rev = df.loc[:, ['Quarter', 'Gross revenue']]
quarterly_rev_total = quarterly_rev.groupby('Quarter').agg({'Gross revenue':'sum'})

# Plot1 - one single plot with two lines
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(monthly_rev.iloc[0:12,0],monthly_rev_total['Gross revenue'],label='Monthly revenue')
ax1.set_xlabel('Month')
ax1.set_ylabel('Gross revenue ($)')

ax2 = ax1.twiny()
ax2.set_xlim([0,4.5])
ax2.set_xlabel('Quarter')
#fig1.xticks(np.arange(min([1,2,3,4]), max([1,2,3,4])+1, 1.0))
ax2.scatter([1,2,3,4],quarterly_rev_total['Gross revenue'],s=50, c='r', marker='x', label='Quarterly reveue')
#ax2.xaxis.set_major_locator(fig1.MaxNLocator(4))
fig1.legend(loc='center right')

#Plot 2 - subplots
fig2 = plt.figure()
fig2, axes = plt.subplots(2, 1)
axes[0].plot(monthly_rev.iloc[0:12,0],monthly_rev_total['Gross revenue'], label='Monthly revenue')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Gross revenue ($)')
axes[0].set_xlim([0,12.5])

axes[1].scatter([1,2,3,4],quarterly_rev_total['Gross revenue'],s=50, c='r', marker='x', label='Quarterly revenue')
axes[1].set_xlabel('Quarter')
axes[1].set_xlim([0,4.5])
axes[1].set_ylabel('Gross revenue ($1)')
fig2.legend(loc='upper right')

#### Part 2 - Cleaning the data

df_clean = df
rental_cost_na = df['Rental cost'].replace(0,np.nan)

# Imputation method 1 - replacing missing values with rental cost mean for each state 
df_clean['Adjusted rent via group means'] = rental_cost_na
df_clean['Adjusted rent via group means'].fillna(df_clean.groupby('State')['Adjusted rent via group means'].transform("mean"), inplace=True)

# Imputation method 2 - Linear regression
# Create a correlation matrix to identify relevant variables (feature selection component)
colormap = plt.cm.RdBu
plt.figure(figsize=(32,10))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(df.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

# This correlation matrix indicates that Gross revenue, Fixed cost and Variable cost are all relevant variables to Rental cost.
# But, they are all highly correlated to each other, suggesting multicollinearity will be a problem.
# Therefore, one possible predictive model is lasso regression to overcome overfit using regularization.

df_clean['Adjusted rent lasso'] = rental_cost_na
df_lr = df_clean[['Gross revenue', 'Variable cost', 'Fixed cost', 'Adjusted rent lasso','Number of products' ]]

# Split dataset into train and test
x_train = df_lr[df_lr['Adjusted rent lasso'].notnull()].drop(columns='Adjusted rent lasso')
y_train = df_lr[df_lr['Adjusted rent lasso'].notnull()]['Adjusted rent lasso']
x_test = df_lr[df_lr['Adjusted rent lasso'].isnull()].drop(columns='Adjusted rent lasso')
y_test = df_lr[df_lr['Adjusted rent lasso'].isnull()]['Adjusted rent lasso']

# Fit the model on the train set
lasso = Lasso(tol=0.0001,max_iter=1e6)
model = lasso.fit(x_train, y_train)

# Predict the missing values
predicted = lasso.predict(x_test)

# Assess performance of model with RMSE
y_true = y_train
y_pred = lasso.predict(x_train)
print(np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

df_clean['Adjusted rent lasso'][df_clean['Adjusted rent lasso'].isnull()] = predicted

# Imputation method 3 - K-nearest neighbors 

df_clean['Adjusted rent knn'] = rental_cost_na
model_knn = KNeighborsRegressor(n_neighbors=3)

# Fit model and predit
model_knn.fit(x_train,y_train)
predicted_knn = model_knn.predict(x_test)

# Assess performance of model
y_true_knn = y_train
y_pred_knn = model_knn.predict(x_train)
print(np.sqrt(metrics.mean_squared_error(y_true_knn,y_pred_knn)))

df_clean['Adjusted rent knn'][df_clean['Adjusted rent knn'].isnull()] = predicted_knn

# Re-calculate rent-adjusted profit margins 
# Model choice = knn 
# Create new variable each location's monthly profit, by substracting total costs from gross revenue
monthly_profit_adj = df_clean.loc[:,'Gross revenue'] - df_clean.loc[:,'Fixed cost'] - df_clean.loc[:,'Variable cost'] - df_clean.loc[:, 'Adjusted rent knn']
df_clean['Adjusted profit'] = monthly_profit_adj

# Calculate annual profit for each location by summing monthly profits
df_annual_profit_adj = df_clean.groupby('Location number').agg({'Adjusted profit':'sum'})

# Calculate mean and median values for annual profit over all stores
mean_profit_adj = df_annual_profit_adj.mean()
median_profit_adj = df_annual_profit_adj.median()
print(mean_profit_adj)
print(median_profit_adj)

# Clearly still not normally distributed.

#### Part 3 - Quantify drivers of success 
# Observation 1 - Effects of Location 

# Annual profits
df_by_state = df_clean.groupby('Location number').agg({'Adjusted profit':'sum', 'State':'max'})
df_by_state_ordered = df_by_state.sort_values('Adjusted profit', ascending=False)
top_performers_annually = df_by_state_ordered.iloc[0:20,:]
bottom_performers_annually = df_by_state_ordered.iloc[313:333,:]
# Monthly profits
df_clean_ordered = df_clean.sort_values('Adjusted profit', ascending=False)
top_performers_monthly = df_clean.iloc[0:200,:]
bottom_performers_monthly = df_clean.iloc[3196:3396,:]

# Calculate group means for profit by state 
state_means_profit = df_clean.groupby('State').agg({'Adjusted profit':'mean'})

df_by_state['State'].value_counts()
top_performers_annually['State'].value_counts()
bottom_performers_annually['State'].value_counts()

# Observation 2 - Seasonality of Performance
monthly_rev1 = df_clean.loc[:, ['Month', 'Adj profit']]
monthly_rev_total1 = monthly_rev1.groupby('Month').agg({'Adj profit':'sum'})
plt.plot([1,2,3,4,5,6,7,8,9,10,11,12],monthly_rev_total)
plt.title('Month vs. Profit')
plt.xlabel('Month')
plt.ylabel('Rent Adjusted Profit ($)')
plt.show()